In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sys
import xarray as xr
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
from netCDF4 import Dataset
from datetime import datetime, timedelta, date
import matplotlib.dates as mdates
import matplotlib.patheffects as PathEffects
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import NaturalEarthFeature

import dask

import geopandas
import regionmask
from matplotlib.path import Path

ERROR 1: PROJ: proj_create_from_database: Open of /home/bwallace/anaconda3/envs/analysis/share/proj failed


In [2]:
from dask.distributed import Client

dask.config.set({'distributed.dashboard.link':'https://jupyter.alcf.anl.gov/theta/user/{USER}/proxy/{port}/status'})

client = Client(n_workers=10,threads_per_worker=1)

print(client.dashboard_link)

from distributed import get_client
worker_client=get_client()

/home/bwallace/anaconda3/envs/analysis/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43935 instead
  warnings.warn(


https://jupyter.alcf.anl.gov/theta/user/bwallace/proxy/43935/status


In [3]:
from scipy.stats import mannwhitneyu
import dask.array as da
from dask import delayed
        
def calc_j_row(ds1,ds2,iterations,i):
    j_results=[]
    for j in range(iterations):
        sp=mannwhitneyu(ds1[:,i,j],ds2[:,i,j])
        j_results.append(sp)
    return j_results

def dask_grid_sig(ds1, ds2, client=worker_client, expected_dims=(15, 899, 1399)): #15,44,69 for coarse; 15,899,1399 for regular
    r"""Performs a grid-to-grid significance test on ds1 and ds2.
    Returns a grid of the same size with p-values from the Mann
    Whitney U test.
    
    Parameters
    ----------
    ds1: (t, y, x) ndarray
        An ndarray in the format of (time, y, x) and shape of expected_dims. 
    ds2: (t, y, x) ndarray
        An ndarray in the format of (time, y, x) and shape of expected_dims.  
    expected_dims: tuple
        The expected shape of ds1 and ds2. Default is (15, 44, 69).
    Returns
    -------
    results: (y, x) ndarray
        Results of significance testing in the form of p-values.
    """    
    
    if ds1.shape == expected_dims and ds2.shape == expected_dims:
        #Since zeros would be < 0.05
        #results = np.ones(shape=(ds1.shape[1], ds1.shape[2]), dtype=float)
        results=[]
        
        future_ds1=client.scatter(ds1)
        future_ds2=client.scatter(ds2)

        for i in range(ds1.shape[1]):
            results.append(delayed(calc_j_row)(future_ds1,future_ds2,ds2.shape[2],i))                    

        return results
    
    else:
        
        raise ValueError("Dimensions are not as expected, given", ds1.shape, "expected", expected_dims)


In [4]:
#resolution options - 10m, 50m, 110m
def draw_geography(ax):
        
    countries_shp = shpreader.natural_earth(resolution='110m',
                                     category='cultural',
                                     name='admin_0_countries')
    
    for country, info in zip(shpreader.Reader(countries_shp).geometries(), 
                             shpreader.Reader(countries_shp).records()):
        if info.attributes['NAME_LONG'] != 'United States':

            ax.add_geometries([country], ccrs.PlateCarree(),
                             facecolor='Grey', edgecolor='k', zorder=6)
            
    lakes_shp = shpreader.natural_earth(resolution='110m',
                                     category='physical',
                                     name='lakes')
    
    for lake, info in zip(shpreader.Reader(lakes_shp).geometries(), 
                             shpreader.Reader(lakes_shp).records()):

        name = info.attributes['name']
        if name == 'Lake Superior' or name == 'Lake Michigan' or \
           name == 'Lake Huron' or name == 'Lake Erie' or name == 'Lake Ontario':
            
            ax.add_geometries([lake], ccrs.PlateCarree(),
                              facecolor='lightsteelblue', edgecolor='k', zorder=6)
            
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '110m', edgecolor='face', 
                                                facecolor='lightsteelblue'), zorder=6)

    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '110m', edgecolor='face', 
                                                facecolor='None'), linewidth=1, zorder=6) 
    
    shapename = 'admin_1_states_provinces_lakes'
    states_shp = shpreader.natural_earth(resolution='110m',
                                     category='cultural', name=shapename)
                                     
    for state, info in zip(shpreader.Reader(states_shp).geometries(), shpreader.Reader(states_shp).records()):
        if info.attributes['admin'] == 'United States of America':

            ax.add_geometries([state], ccrs.PlateCarree(),
                              facecolor='grey', linewidth=0.5,edgecolor='k')
            
    for state, info in zip(shpreader.Reader(states_shp).geometries(), shpreader.Reader(states_shp).records()):
        if info.attributes['admin'] == 'United States of America':

            ax.add_geometries([state], ccrs.PlateCarree(),
                              facecolor='None', linewidth=0.5,edgecolor='k', zorder=6)

    return ax

In [5]:
states_provinces = NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none'
)

In [6]:
wrf_native_coords=xr.open_dataset('../data/geog_info.nc')
wrf_lat=wrf_native_coords.XLAT_M[0]
wrf_lon=wrf_native_coords.XLONG_M[0]

#wrf_lat=wrf_lat.coarsen(south_north=20,boundary='trim').mean().coarsen(west_east=20,boundary='trim').mean()
#wrf_lon=wrf_lon.coarsen(south_north=20,boundary='trim').mean().coarsen(west_east=20,boundary='trim').mean()

In [8]:
path = '../data/'

dc = {}

for couple in ['All', 'Unforced', 'Forced']:
    for season in ['ALL', 'MAM', 'DJF', 'JJA', 'SON']:
        for sim in ['historical', 'end_of_century_8p5']:
            dc[sim+couple+season] = xr.open_dataset(path + sim + '/MCS_rain_'+couple+'_'+sim+'_'+season+'.nc')
        
        dc['diff'+couple+season] = dc['end_of_century_8p5'+couple+season]['__xarray_dataarray_variable__'].mean('water_year') - dc['historical'+couple+season]['__xarray_dataarray_variable__'].mean('water_year')
        
        dc['sig'+couple+season]=dask_grid_sig(dc['historical'+couple+season]['__xarray_dataarray_variable__'].values,dc['end_of_century_8p5'+couple+season]['__xarray_dataarray_variable__'].values)
        dc['sig'+couple+season]=dask.compute(dc['sig'+couple+season])[0]
        dc['sig'+couple+season]=np.array(dc['sig'+couple+season])[:,:,1]

2024-05-02 20:43:27,504 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:43:28,831 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:43:30,497 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:43:33,608 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:43:41,605 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:43:45,947 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:43:49,354 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-05-02 20:45:09,091 - distributed.utils_perf - WARNING - full garbage collections took

In [ ]:
# Set some projections for our data (Plate Carree)
# and output maps (Lambert Conformal)

# Data projection;
dataproj = ccrs.PlateCarree()

# Plot projection
# The look you want for the view, LambertConformal for mid-latitude view
plotproj = ccrs.LambertConformal(central_longitude=-95.,
                                 central_latitude=40.,
                                 standard_parallels=[30, 60])

In [ ]:
reg_cmap=mpl.colormaps['YlGnBu'].resampled(10)

diff_cmap=mpl.colormaps['BrBG'].resampled(10)

plt.rcParams['hatch.color'] = 'dimgrey'

In [ ]:
letters = ['a.', 'b.', 'c.', 'd.', 'e.', 'f.', 'g.', 'h.', 'i.', 'j.', 'k.', 'l.', 'm.', 'n.', 'o.', 'p.', 'q.', 'r.', 's.', 't.', 'u.', 'v.']

force_categories = ['All', 'Forced', 'Unforced']

season_categories = ['Annual', 'DJF', 'MAM', 'JJA', 'SON']

In [ ]:
seasons = ['ALL', 'DJF', 'MAM', 'JJA', 'SON']

fig=plt.figure(constrained_layout=True, figsize=(15,10))
gs=fig.add_gridspec(3,5)

for i in range(15):
    ax=fig.add_subplot(gs[i//5, i%5], projection=plotproj)
    ax.set_extent([-101.5, -73, 24., 53.], ccrs.PlateCarree())

    if i%5==0:
        c_annual = ax.pcolormesh(wrf_lon, wrf_lat, dc['historical'+force_categories[i//5]+seasons[i%5]]['__xarray_dataarray_variable__'].mean('water_year'), vmin=0, vmax=800, cmap=reg_cmap, zorder=5, transform=dataproj)
    else:
        c_seasonal = ax.pcolormesh(wrf_lon, wrf_lat, dc['historical'+force_categories[i//5]+seasons[i%5]]['__xarray_dataarray_variable__'].mean('water_year'), vmin=0, vmax=300, cmap=reg_cmap, zorder=5, transform=dataproj)


    if i%5==0 and i//5==0:
        cbar_annual = []
    elif i%5==1 and i//5==0:
        cbar_seasonal = []

    if i%5==0:
        cbar_annual.append(ax)
    else:
        cbar_seasonal.append(ax)

    if i%5==0 and i//5==2:
        cbar=plt.colorbar(c_annual,ax=cbar_annual,orientation='horizontal',aspect=25,fraction=0.01,pad=0.02,
                         ticks=[0, 240, 560, 800])
        cbar.set_label('Annual Average MCS Rain (mm)',fontsize=12)
        cbar.ax.tick_params(labelsize=12)
    elif i%5==4 and i//5==2:
        cbar=plt.colorbar(c_seasonal,ax=cbar_seasonal,orientation='horizontal',aspect=100,fraction=0.01,pad=0.02,
                         ticks=[0, 60, 120, 180, 240, 300])
        cbar.set_label('Seasonal Average MCS Rain (mm)',fontsize=12)
        cbar.ax.tick_params(labelsize=12)



    ax = draw_geography(ax)

    t=ax.text(0.03,0.92, letters[i]+' '+force_categories[i//5]+' '+season_categories[i%5]+' MCS',size=12,color='k',\
              horizontalalignment='left',\
              path_effects=[PathEffects.withStroke(linewidth=4,foreground='white')],\
              transform=ax.transAxes,zorder=15)
    t.set_bbox(dict(facecolor='white',alpha=0.5,edgecolor='black'))

    ll=ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle=':',lw=1.2,zorder=11)
    ll.top_labels=False
    ll.right_labels=False
    
    if i%5>0:
        ll.left_labels=False
    else:
        ll.left_labels=True

    if i//5<2:
        ll.bottom_labels=False
    else:
        ll.bottom_labels=True
        
    ll.xlocator = mticker.FixedLocator([-120,-110,-100,-90,-80,-70])
    ll.rotate_labels=True
    ll.xlabel_style = {'rotation':45,  'size':9}
    ll.ylabel_style = {'rotation':15, 'size':9}


plt.show()
plt.close()

In [ ]:
seasons = ['ALL', 'DJF', 'MAM', 'JJA', 'SON']

fig=plt.figure(constrained_layout=True, figsize=(15,10))
gs=fig.add_gridspec(3,5)

for i in range(15):
    ax=fig.add_subplot(gs[i//5, i%5], projection=plotproj)
    ax.set_extent([-101.5, -73, 24., 53.], ccrs.PlateCarree())

    if i%5==0:
        ax.pcolormesh(wrf_lon, wrf_lat, dc['diff'+force_categories[i//5]+seasons[i%5]], vmin=-300, vmax=300, cmap=diff_cmap, zorder=5,transform=dataproj)
        plt.fill_between([-1e10,1e10],-1e10,1e10,hatch='////',color='none',edgecolor='k', alpha=0.5, zorder=5)
        c_annual = ax.pcolormesh(wrf_lon, wrf_lat, dc['diff'+force_categories[i//5]+seasons[i%5]].where(dc['sig'+force_categories[i//5]+seasons[i%5]]>0.05), vmin=-300, vmax=300, cmap=diff_cmap, zorder=5,transform=dataproj)
    else:
        ax.pcolormesh(wrf_lon, wrf_lat, dc['diff'+force_categories[i//5]+seasons[i%5]], vmin=-200, vmax=200, cmap=diff_cmap, zorder=5,transform=dataproj)
        plt.fill_between([-1e10,1e10],-1e10,1e10,hatch='////',color='none',edgecolor='k', alpha=0.5, zorder=5)
        c_seasonal = ax.pcolormesh(wrf_lon, wrf_lat, dc['diff'+force_categories[i//5]+seasons[i%5]].where(dc['sig'+force_categories[i//5]+seasons[i%5]]>0.05), vmin=-200, vmax=200, cmap=diff_cmap, zorder=5,transform=dataproj)


    if i%5==0 and i//5==0:
        cbar_annual = []
    elif i%5==1 and i//5==0:
        cbar_seasonal = []

    if i%5==0:
        cbar_annual.append(ax)
    else:
        cbar_seasonal.append(ax)

    if i%5==0 and i//5==2:
        cbar=plt.colorbar(c_annual,ax=cbar_annual,orientation='horizontal',aspect=25,fraction=0.01,pad=0.02,
                         ticks=[-300, -150, 0, 150, 300])
        cbar.set_label('$\Delta$ Annual Average MCS Rain (mm)',fontsize=12)
        cbar.ax.tick_params(labelsize=12)
    elif i%5==4 and i//5==2:
        cbar=plt.colorbar(c_seasonal,ax=cbar_seasonal,orientation='horizontal',aspect=100,fraction=0.01,pad=0.02,
                         ticks=[-200, -150, -100, -50, 0, 50, 100, 150, 200])
        cbar.set_label('$\Delta$ Seasonal Average MCS Rain (mm)',fontsize=12)
        cbar.ax.tick_params(labelsize=12)



    ax = draw_geography(ax)

    t=ax.text(0.03,0.92, letters[i]+' '+force_categories[i//5]+' '+season_categories[i%5]+' MCS',size=12,color='k',\
              horizontalalignment='left',\
              path_effects=[PathEffects.withStroke(linewidth=4,foreground='white')],\
              transform=ax.transAxes,zorder=15)
    t.set_bbox(dict(facecolor='white',alpha=0.5,edgecolor='black'))

    ll=ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle=':',lw=1.2,zorder=11)
    ll.top_labels=False
    ll.right_labels=False
    
    if i%5>0:
        ll.left_labels=False
    else:
        ll.left_labels=True

    if i//5<2:
        ll.bottom_labels=False
    else:
        ll.bottom_labels=True
        
    ll.xlocator = mticker.FixedLocator([-120,-110,-100,-90,-80,-70])
    ll.rotate_labels=True
    ll.xlabel_style = {'rotation':45,  'size':9}
    ll.ylabel_style = {'rotation':15, 'size':9}

plt.show()
plt.close()